In [1]:
#in the name of god

In [2]:
import pandas as pd 
import numpy as np
from IPython.display import clear_output
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import matplotlib.cm as cm
import matplotlib as matplotlib

from fastdtw import fastdtw

def color_map_color(value, cmap_name='coolwarm', vmin=0, vmax=10):
    # norm = plt.Normalize(vmin, vmax)
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)  # PiYG
    rgb = cmap(norm(abs(value)))[:3]  # will return rgba, we take only first 3 so we get rgb
    color = matplotlib.colors.rgb2hex(rgb)
    return color
cl=['r','g','b','c','m','y','k']
color=dict()
for i,el in enumerate(cl):
    color.update({i:el})
#extract the labels for clutering precision its for after clustering
def time_convertor(x):
    s,h,m=0,0,0
    s=round(x%60,2)
    m=int(x/60)
    h=int(m/60)
    m=m%60
    time=str(h)+':'+str(m)+':'+str(s)
    return pd.to_datetime('1970-01-01 '+time)


# Display figures inline in Jupyter notebook
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 5)})

In [3]:
with open("normalized_data.txt", "rb") as fp:
        dfRaw = pickle.load(fp)
        
def get_label(number,df):
    label_event_lenght=dict()
    df=pd.read_csv(f'{number}\\groundTruth.csv')
    df['length']=df.en-df.st
    for event , dft in df.groupby('evento'):
        if (event in label_event_lenght):
            label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
        else:
            label_event_lenght.update({event : list(dft.length)})

    df['st_time']=df.apply(lambda x:time_convertor(x.st) ,axis=1 )
    df['en_time']=df.apply(lambda x:time_convertor(x.en) ,axis=1 )
    df=df.drop('st',1).drop('en',1).drop('length',1)
    evented_label=list()
    for i in range(len(df)):
        evented_label.append((df.iloc[i][0],df.iloc[i][1],df.iloc[i][2]))
    eve=pd.DataFrame(evented_label)
    eve[3]=[f'evented_label_{number}' for x in evented_label]
    return eve
Ev=[get_label(dfRaw[x][0],dfRaw[x][1]) for x in range(len(dfRaw))]
evented_label_All=pd.concat(Ev).reset_index().drop('index',1)


In [4]:
rate=(dfRaw[0][1].index[11]-dfRaw[0][1].index[10]).total_seconds()

print(f"rate is :{rate}")
pd.DataFrame([(x[0],round(((x[2]-x[1]).total_seconds())/rate)) for x in evented_label_All.iloc]).groupby(0).agg({1:"max"})

rate is :0.02


,1
0,
aceleracao_agressiva,245
curva_direita_agressiva,225
curva_esquerda_agressiva,225
evento_nao_agressivo,225
freada_agressiva,185
troca_faixa_direita_agressiva,125
troca_faixa_esquerda_agressiva,120


In [5]:
#supervised
window={226:['curva_direita_agressiva','curva_esquerda_agressiva','aceleracao_agressiva'],
        126:['troca_faixa_direita_agressiva','troca_faixa_esquerda_agressiva'],
        186:['freada_agressiva']
       }

In [6]:
#Dont RUN this part
#we should optimize M
#lengh of random event
import random
random.seed(123)
def TempEvent_genarator(df,L,i):
    temp_event=list()
    pointer=0
    stop=False
    #go on timeserise for event extraction
    while stop!=True:
        M=round(random.gauss(0.05,1)*L)
        clear_output(wait=True)
        pointer=pointer + M
        l_min=pointer-int(L/2)
        l_max=pointer+int(L/2)
        if l_max<0:
            l_max=L/2
        if (l_min<0):
            l_min=0
        if (l_max>len(df)):
            l_max=len(df)
            stop=True
        print('extraction ',l_max/len(df),' % of ',i,' is complete')
        temp_event.append(df[int(l_min):int(l_max)])
    return temp_event

temp_event=list()
for l,DF in enumerate([x[1] for x in dfRaw]):
    te=list()
    for i,lw in enumerate(window):
        te.append((window[lw],TempEvent_genarator(DF,lw,i+1)))
    temp_event.append(te)

with open("temp_event_new_005.txt", "wb") as fp:
    pickle.dump(temp_event, fp)

extraction  1.0  % of  3  is complete


In [7]:
with open("temp_event_new_005.txt", "rb") as fp:
    temp_event = pickle.load(fp)

In [8]:
def has_overlap(A_start, A_end, B_start, B_end):
    latest_start = max(A_start, B_start)
    earliest_end = min(A_end, B_end)
    return latest_start <= earliest_end

def export_labels(tempevent,evented_label):
    EvL=pd.DataFrame(evented_label)
    Labels={
          226:[],
          126:[],
          186:[],
           }
    for i,wind in enumerate(Labels):
        label=['NAG' for x in tempevent[i][1]]
        for name in tempevent[i][0]:
            el=[(x[1],x[2]) for x in EvL.iloc if x[0]==name]
            for j,te in enumerate(tempevent[i][1]):
                    if len(te)!=0:
                        sta =te.index[0]
                        ena =te.index[-1]
                        for stb,enb in el:
                            if has_overlap(sta,ena,stb,enb):
                                label[j]=name
                    else:
                        label[j]="NA"
                        
        Labels[wind]=label
    return Labels


Data={
      226:[],
      126:[],
      186:[],
       }
for i,temp in enumerate(temp_event):
    TempEv={
              226:[ x for x in temp_event[i][0][1] ],
              126:[ x for x in temp_event[i][1][1] ],
              186:[ x for x in temp_event[i][2][1] ],
               }
    Label=export_labels(temp_event[i],Ev[i])
    #merge together
    for wind in Data:
        for j in range(len(TempEv[wind])):
            Lb=Label[wind][j]
            if Lb != 'NA':
                Data[wind].append((Lb,TempEv[wind][j]))


In [9]:
#dont run this part
#calculate dist of Data from labdeled event
t1=time.time()

k,n=0,0
num_event=dict(evented_label_All.groupby(0).size())
for lenght in window:
    for name in window[lenght]:
        k=k+len(Data[lenght])*num_event[name]

Data_DS={
      226:[],
      126:[],
      186:[],
       }

for wind in Data:
    for label,event in Data[wind]:
         #---------for each Data-distance from events----------
        Y=list()
        for from_event,dfe in evented_label_All.groupby(0):
            if from_event in window[wind]:
                X=list()
                #____for each event_distance from each label_____
                for el in dfe.iloc:
                    clear_output(wait=True)
                    n=n+1
                    print('disstance calculatoin ',round((n/k)*100,4),' %  is complete')
                    if   el[3]=='evented_label_16':
                        X.append(fastdtw(event,dfRaw[0][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_17':
                        X.append(fastdtw(event,dfRaw[1][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_20':
                        X.append(fastdtw(event,dfRaw[2][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_21':
                        X.append(fastdtw(event,dfRaw[3][1][el[1]:el[2]])[0])
                #_________________________________________________
                Y.append((from_event,X))
        Data_DS[wind].append((label,Y))
        #------------------------------------------------------

t2=time.time()
with open("Disstance_new_005.txt", "wb") as fp:
    pickle.dump(Data_DS, fp)

print(round((t2-t1)/60) ,'min has time for calculaton distant')

disstance calculatoin  100.0  %  is complete
1322 min has time for calculaton distant


In [ ]:
with open("Disstance_new_005.txt", "rb") as fp:
    Data_DS = pickle.load(fp)

In [10]:
#prepare dataset for labeling
TrainData=dict()
for win in Data_DS:
    x=Data_DS[win]
    dumy_list=list()
    for events in x:
        dumy_dict=dict()
        dumy_dict.update({'label':events[0]})
        for name , event in events[1]:
            for tag,number in enumerate(event):
                dumy_dict.update({name+str(tag):number})
        dumy_list.append(dumy_dict)
    TrainData.update({win:pd.DataFrame(dumy_list)})

In [11]:
TrainData[226].head(10)

,label,aceleracao_agressiva0,aceleracao_agressiva1,aceleracao_agressiva2,aceleracao_agressiva3,aceleracao_agressiva4,aceleracao_agressiva5,aceleracao_agressiva6,aceleracao_agressiva7,aceleracao_agressiva8,...,curva_esquerda_agressiva1,curva_esquerda_agressiva2,curva_esquerda_agressiva3,curva_esquerda_agressiva4,curva_esquerda_agressiva5,curva_esquerda_agressiva6,curva_esquerda_agressiva7,curva_esquerda_agressiva8,curva_esquerda_agressiva9,curva_esquerda_agressiva10
0,NAG,169.722455,177.909264,189.157378,179.027278,210.836897,196.652428,190.543919,214.707394,179.881763,...,136.697395,132.303529,127.355028,124.763061,142.949748,129.111795,179.528992,180.184488,142.541302,128.246754
1,NAG,174.367035,186.634108,195.792538,181.028956,209.684914,194.977302,196.650909,214.299985,185.699722,...,137.605357,128.887522,128.322159,125.520809,147.648039,135.682084,184.209117,177.232556,151.046312,138.438405
2,NAG,144.015236,155.200913,185.611280,163.425143,211.835726,197.120401,163.953364,214.991170,152.992899,...,132.703006,129.031549,127.928300,126.528970,134.009431,114.905350,164.329986,163.091794,125.469962,112.669794
3,NAG,174.092715,186.632822,195.521537,181.576146,209.369322,195.600331,197.167643,214.607224,186.414400,...,137.348702,129.868907,128.537179,125.079870,147.488938,134.596282,184.398242,178.132335,151.131984,136.814033
4,NAG,181.689673,193.428311,200.599339,178.997268,207.086366,193.625956,194.665944,211.654300,184.079498,...,142.298572,131.763653,132.223386,131.519946,149.632223,145.955439,184.806101,174.577237,148.854275,142.387598
5,NAG,179.510354,190.426508,198.374715,179.553567,208.396064,194.944210,195.347306,212.583807,184.089436,...,139.796722,129.850783,129.095628,127.800960,148.006250,140.906378,183.983581,175.099483,150.811955,141.506472
6,NAG,118.092895,155.951069,188.535101,158.334436,209.275921,194.866503,157.645599,209.746796,106.860179,...,129.176752,123.409047,126.361287,122.151186,129.821411,101.881771,154.109689,145.774343,112.365762,99.144026
7,NAG,116.767994,154.012840,186.910347,157.659993,208.636952,194.138228,156.684596,208.761775,105.522517,...,128.446790,122.793786,125.769072,121.460850,128.818569,100.675347,153.314173,144.562271,111.443185,98.077169
8,NAG,130.634924,153.310721,184.271271,163.927164,213.623652,198.463137,162.443019,214.371831,137.272694,...,133.601551,128.690776,129.607363,126.537187,131.756769,106.913405,159.528204,153.231225,117.468305,104.637897
9,NAG,116.543276,153.758608,186.775919,157.586505,208.541979,194.013138,156.565187,208.611255,105.331526,...,128.428508,122.947856,125.727307,121.644667,128.691201,100.532003,153.391320,144.555611,111.291329,97.923520


In [12]:
#normalize Train Data
#Balance it 

In [13]:
#dont run this part
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report
t1=time.time()
n,k=0,12*10*3

Result={
      226:[],
      126:[],
      186:[],
       }

for wid in Result:
    data=TrainData[wid].drop('label',1)
    label=TrainData[wid].label
    #
    for Max_depth in range(1,13):
        estimator = Id3Estimator(max_depth=Max_depth, min_samples_split=1, prune=True,
                        gain_ratio=True, min_entropy_decrease=0, is_repeating=True)
        #run it in itaraion for the best trees
        NUMBER_OF_TEST=10
        temp=list()
        for NT in range(NUMBER_OF_TEST):
            clear_output(wait=True)
            n=n+1
            print('disstance calculatoin ',round((n/k)*100,4),' %  is complete')
            estimator.fit(data,label , check_input=True)
            ACC=accuracy_score(y_true=label,y_pred=estimator.predict(data))
            temp.append((ACC,deepcopy(estimator)))
        temp_index=[(x[0],i) for i,x in enumerate(temp)]
        temp_index.sort()
        Acc=temp[temp_index[-1][1]][0]
        estimator=temp[temp_index[-1][1]][1]
        Result[wid].append((Acc,deepcopy(estimator)))

with open("Model_new_005.txt", "wb") as fp:
    pickle.dump(Result, fp)
t2=time.time()
print(round((t2-t1)/60) ,'min has time for calculaton Model')

disstance calculatoin  100.0  %  is complete
306 min has time for calculaton Model


In [20]:
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report

with open("Model_new_005.txt", "rb") as fp:
    Result = pickle.load(fp)

In [14]:
Max_depth=7
wid=126

label=TrainData[wid].label
data=TrainData[wid].drop('label',1)

Acc=Result[wid][Max_depth-1][0]
estimator=Result[wid][Max_depth-1][1]
print(Acc)
print(classification_report(label, estimator.predict(data)))
#print(export_text(estimator.tree_))

0.9903763286411951
                                precision    recall  f1-score   support

                           NAG       0.99      1.00      1.00     20490
 troca_faixa_direita_agressiva       0.97      0.53      0.69       245
troca_faixa_esquerda_agressiva       1.00      0.45      0.62       151

                      accuracy                           0.99     20886
                     macro avg       0.99      0.66      0.77     20886
                  weighted avg       0.99      0.99      0.99     20886



In [15]:
Max_depth=7
wid=186

label=TrainData[wid].label
data=TrainData[wid].drop('label',1)

Acc=Result[wid][Max_depth-1][0]
estimator=Result[wid][Max_depth-1][1]
print(Acc)
print(classification_report(label, estimator.predict(data)))
#print(export_text(estimator.tree_))

0.9879884025956096
                  precision    recall  f1-score   support

             NAG       0.99      1.00      0.99     14038
freada_agressiva       0.92      0.67      0.77       448

        accuracy                           0.99     14486
       macro avg       0.96      0.83      0.88     14486
    weighted avg       0.99      0.99      0.99     14486



In [16]:
Max_depth=11
wid=226

label=TrainData[wid].label
data=TrainData[wid].drop('label',1)

Acc=Result[wid][Max_depth-1][0]
estimator=Result[wid][Max_depth-1][1]
print(Acc)
print(classification_report(label, estimator.predict(data)))
#print(export_text(estimator.tree_))

0.9767716867978714
                          precision    recall  f1-score   support

                     NAG       0.98      1.00      0.99     10733
    aceleracao_agressiva       0.96      0.70      0.81       521
 curva_direita_agressiva       0.99      0.92      0.95       422
curva_esquerda_agressiva       0.95      0.65      0.77       163

                accuracy                           0.98     11839
               macro avg       0.97      0.82      0.88     11839
            weighted avg       0.98      0.98      0.98     11839

